<a href="https://colab.research.google.com/github/dhruv20047228/LLM/blob/main/coding_attention_mechanism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Coding Attention Mechanism**

In [ ]:
from importlib.metadata import version
print("torch version:", version("torch"))

In [ ]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [ ]:
#Step 1 : computation of the unormalized attention scores with query x2
query = inputs[1]
attn_scores_2 = torch.empty(inputs.shape[0])
for i,x_i in enumerate(inputs):
  attn_scores_2[i] = torch.dot(x_i, query)

print(attn_scores_2)

In [ ]:
res = 0
for idx, element in enumerate(inputs[0]):
  res += inputs[0][idx] * query[idx]

  print(res)
  print(torch.dot(inputs[0], query))

In [ ]:
#Step 2: Normalization of unnormalized attention scores
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()

print("Attention weights:", attn_scores_2)
print("Sum:", attn_weights_2_tmp.sum())

In [ ]:
def softmax_naive(x):
  return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)

print("Attention weights:", attn_weights_2_naive)
print("Sum: ", attn_weights_2_naive.sum())

In [ ]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print("Attention weights:", attn_weights_2)
print("Sum: ", attn_weights_2.sum())

In [ ]:
#Computation of context vector z2
query = inputs[1]

context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
  context_vec_2 += attn_weights_2[i]*x_i

print(context_vec_2)


#**Computation of attention weights and context vector**

In [ ]:
#Computation of the unnormalized attention score
attn_scores = torch.empty(6,6)

for i, x_i in enumerate(inputs):
  for j, x_j in enumerate(inputs):
    attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

In [ ]:
#same calculations as above but by matrix multiplications
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

In [ ]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

In [ ]:
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("Row 2 sum:", row_2_sum)

print("All row sums:", attn_weights.sum(dim=-1))

In [ ]:
#Computation of context vector
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

In [ ]:
print("Previous 2nd context vector:", context_vec_2)

#**Computation of attention weights in original transformer architecture**

In [ ]:
import torch
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2


In [ ]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.randn(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.randn(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.randn(d_in, d_out), requires_grad=False)

In [ ]:
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

print(query_2)

In [ ]:
keys = inputs @ W_key
values = inputs @ W_value

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

In [ ]:
keys_2 = keys[1]
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

In [ ]:
attn_scores_2 = query_2 @ keys.T
print(attn_scores_2)

In [ ]:
#we are scaling attention scores by dividing them with squre root of embedding dimension
d_k = keys.shape[1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

In [ ]:
#final context vectos is achieved by matrix multiplication pf final attention scores
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

#**Implementing compact self attention class**

In [ ]:
import torch.nn  as nn

class SelfAttention_v1(nn.Module):

  def __init__(self, d_in, d_out):
    super().__init__()
    self.W_query = nn.Parameter(torch.rand(d_in,d_out))
    self.W_key = nn.Parameter(torch.rand(d_in,d_out))
    self.W_value = nn.Parameter(torch.rand(d_in,d_out))

  def forward(self, x):
    keys = x @ self.W_key
    queries = x @ self.W_query
    values = x @ self.W_value

    attn_scores = queries @ keys.T #omega
    attn_weights = torch.softmax(
        attn_scores / keys.shape[-1]**0.5, dim=-1
    )

    context_vec = attn_weights @ values
    return context_vec

torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))
